In [ ]:
from astra import astra, tools
import os, errno

In [ ]:
MY_ASTRA_BIN = os.path.expandvars('$HOME/Code/astra/Astra')
MY_WORKDIR = os.getcwd()+'/work/'
MY_TEMPLATE = os.path.expandvars('$HOME/Code/GitHub/lume-astra/templates/dcgun/astra.in')
tools.mkdir_p(MY_WORKDIR)
os.path.exists(MY_ASTRA_BIN), os.path.exists(MY_TEMPLATE), os.path.exists(MY_WORKDIR)

In [ ]:
a = astra.Astra(astra_bin=MY_ASTRA_BIN, input_file=MY_TEMPLATE, workdir = 'work')

In [ ]:
a.run_astra()

In [ ]:
a.output

In [ ]:
a.sim_input_file

In [ ]:
a.sim_path

In [ ]:
replacements = {'MaxE(1)': -10, 'MaxB(1)':1.0}

In [ ]:
tools.replace_parameters(a.source_input_file,a.sim_input_file, replacements, delims='=', commentchar = '#', warn=True)

In [ ]:
a.run_astra(verbose=False)
a.output

In [ ]:
a.clean()